In [3]:
import re
import requests
from bs4 import BeautifulSoup
import time
from urllib import parse
import json
from selenium import webdriver

import datetime
import ssl

# 영화 제목 
# 일시 
# 장소 


#     ##### 동대문구 크롤러##### (월별)
def dongdaemungu_movie_crawler(libGroup):
    
    params = {
        'libGroup': libGroup,
    }
    url = "http://www.l4d.or.kr/library/index.php?g_page=culture&m_page=culture04&" + parse.urlencode(params)

    
    request = requests.get(url)
    response = request.text
    soup = BeautifulSoup(response, 'lxml')
    
    # 영화별 정보 담은 박스 
    data_boxes = soup.select('div.data_wrapper')

    for data_box in data_boxes:
        
        # 사진 뽑기
        pic_url = data_box.select_one('div.pic > a > img').get('src')
        # 사진 url을 뽑아보니 두가지 경우가 있다. 
        # http로 시작 하는 경우, 상대결로로서 l4d주소 있다 가정하고 뒷주소만 있는경우
        
        # 뒷 주소만 있는경우 ../경로표시 빼고 l4d 주소 더해준다. 
        if not re.findall('http://',pic_url):
            pic_url =  'http://www.l4d.or.kr/'+ re.findall('\..(.*)',pic_url)[0]
        
        #@ 사진
        print(f'pic_url: {pic_url}')
        
        #제목 뽑기 
        title = data_box.select_one('h3.recom_title > a').get_text(strip=True)
        #@ 제목 
        print(f'title: {title}')
        
        # 나머지 정보들 뽑기.
        
#         print(data_box.prettify())
        
#         <li>
#           <span class="fb">
#            일시
#           </span>
#           : 2018.09.02 13:00 (일)
#          </li>
#
#          이런식으로 리스트들이 들어있는데 
#          li를 자식인 span.fb의 text인 일시 를 기준으로 찾고 싶다. 
#          즉 기준이되는 찾은 테그의 부모의 택스트를 가져오고 싶은것.
        
        # 여기서 선택 된 것은 tag리스트이고 
        # 이중 text가 일시 인것을 먼저 찾아보겠다. 
        

        span_tags = data_box.select('ul > li > span.fb')
#         print(span_tags)

        when = ""#일시
        place = ""#장소
        runtime = ""#시간        # 만약에 해당 항목 없으면 ""
        
        for span_tag in span_tags:
            span_tag_text = span_tag.get_text(strip=True)
            
            if span_tag_text == "일시":
                when = span_tag.parent.get_text(strip=True)
                #@ 일시
                print(when)
            
            elif span_tag_text =="장소":
                place = span_tag.parent.get_text(strip=True)
                #@ 장소 
                print(place)
                
            elif span_tag_text =="시간":
                runtime = span_tag.parent.get_text(strip=True)
                #@ 시간
                print(runtime)
                
                
        print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')

        
        
#    ##### 성동구 크롤러 ##### (페이지별)

def seongdonggu_movie_crawler(area_code,year):
    
    context = ssl._create_unverified_context()
    
    # 페이지 1~5를 탐색해서 해당 월에 상영하는 작품만 가져오자.
    for page in range(1,6):
        
        params = {
            'page':page
        }
        url = "https://www.sdlib.or.kr/"+ area_code + "/U07080.asp?viewtype=" + parse.urlencode(params)
        
#         url = "https://www.sdlib.or.kr/SD/U07080.asp?viewtype=&page=2&libno="
        
#         result = urlopen(url,context=context)
#         bsobj = BeautifulSoup(result.read(),'lxml')
#         print(bsobj)
                         
        request = requests.get(url)
        response = request.text
        soup = BeautifulSoup(response, 'lxml')

        print(soup)
    
def main_movie_crawler():
    
    # 오늘 날짜 먼저 가져옴 
    now = datetime.datetime.now()
    year = now.year
    day = now.day
    month = now.month

    
#     ##### 동대문구 크롤러##### 

#     dongdaemungu_area_code_list = ['MA','MF','MB','MC','MJ'] 
#     # 정보화,답십리, 장안, 용두, 휘셩 (이문 제외)
# #     dongdaemungu_area_code_list = ['MA']
    
#     for libGroup in dongdaemungu_area_code_list:
#         dongdaemungu_movie_crawler(libGroup)
#         print(f'{libGroup} #####################################################################################################################################')


    ##### 광진구 크롤러 #####
    
#     seongdonggu_area_code_list = ['SD','YD','SS','KH','CG'] 
    # 성동구립,용답, 장안, 성수, 금호,청계
    seongdonggu_area_code_list = ['SD']
    
    for area_code in seongdonggu_area_code_list:
        seongdonggu_movie_crawler(area_code,year)
        
    



    

In [4]:
main_movie_crawler()

SSLError: HTTPSConnectionPool(host='www.sdlib.or.kr', port=443): Max retries exceeded with url: /SD/U07080.asp?viewtype=page=1 (Caused by SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)'),))

In [ ]:
위에서 성동구 크롤러 오류 떠서 
HTTPSConnectionPool : Max retries exceeded with url:
        이런.....
        
아래에서 스텍오버 플로 참고해서 다시 해봄.

In [70]:
import re
import requests
from bs4 import BeautifulSoup
import time
from urllib import parse
import json
from selenium import webdriver

import datetime

import ssl
import urllib.request as req
    



# 영화 제목 
# 일시 
# 장소 


#     ##### 동대문구 크롤러##### (월별)
def dongdaemungu_movie_crawler(libGroup):
    
    params = {
        'libGroup': libGroup,
    }
    url = "http://www.l4d.or.kr/library/index.php?g_page=culture&m_page=culture04&" + parse.urlencode(params)

    
    request = requests.get(url)
    response = request.text
    soup = BeautifulSoup(response, 'lxml')
    
    # 영화별 정보 담은 박스 
    data_boxes = soup.select('div.data_wrapper')

    for data_box in data_boxes:
        
        # 사진 뽑기
        pic_url = data_box.select_one('div.pic > a > img').get('src')
        # 사진 url을 뽑아보니 두가지 경우가 있다. 
        # http로 시작 하는 경우, 상대결로로서 l4d주소 있다 가정하고 뒷주소만 있는경우
        
        # 뒷 주소만 있는경우 ../경로표시 빼고 l4d 주소 더해준다. 
        if not re.findall('http://',pic_url):
            pic_url =  'http://www.l4d.or.kr/'+ re.findall('\..(.*)',pic_url)[0]
        
        #@ 사진
        print(f'pic_url: {pic_url}')
        
        #제목 뽑기 
        title = data_box.select_one('h3.recom_title > a').get_text(strip=True)
        #@ 제목 
        print(f'title: {title}')
        
        # 나머지 정보들 뽑기.
        
#         print(data_box.prettify())
        
#         <li>
#           <span class="fb">
#            일시
#           </span>
#           : 2018.09.02 13:00 (일)
#          </li>
#
#          이런식으로 리스트들이 들어있는데 
#          li를 자식인 span.fb의 text인 일시 를 기준으로 찾고 싶다. 
#          즉 기준이되는 찾은 테그의 부모의 택스트를 가져오고 싶은것.
        
        # 여기서 선택 된 것은 tag리스트이고 
        # 이중 text가 일시 인것을 먼저 찾아보겠다. 
        

        span_tags = data_box.select('ul > li > span.fb')
#         print(span_tags)

        when = ""#일시
        place = ""#장소
        runtime = ""#시간        # 만약에 해당 항목 없으면 ""
        
        for span_tag in span_tags:
            span_tag_text = span_tag.get_text(strip=True)
            
            if span_tag_text == "일시":
                when = span_tag.parent.get_text(strip=True)
                #@ 일시
                print(when)
            
            elif span_tag_text =="장소":
                place = span_tag.parent.get_text(strip=True)
                #@ 장소 
                print(place)
                
            elif span_tag_text =="시간":
                runtime = span_tag.parent.get_text(strip=True)
                #@ 시간
                print(runtime)
                
                
        print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')

        
        
#    ##### 성동구 크롤러 ##### (페이지별)

def seongdonggu_movie_crawler(area_code,year):

    
    # 페이지 1~5를 탐색해서 해당 월에 상영하는 작품만 가져오자.
    for page in range(1,6):
        
        params = {
            'page':page
        }

        url = "https://www.sdlib.or.kr/"+ area_code + "/U07080.asp?viewtype=list&" + parse.urlencode(params)
#     url = "https://www.sdlib.or.kr/SD/U07080.asp?viewtype=list&page=1&libno="

        ssl._create_default_https_context = ssl._create_unverified_context
        res = req.urlopen(url)
        soup = BeautifulSoup(res, 'lxml')

        table_rows = soup.select('table.table700 > tbody > tr')


        for row in table_rows:
            for index, td in enumerate(row.select('td')):
                if index == 0:
                    print(f'time_date : {td.get_text(strip=True)}') #날짜 
                elif index ==2:
                    print(f'time_hour : {td.get_text(strip=True)}') # 시작시간 
                elif index ==3:
                    print(f'title : {td.get_text(strip=True)}') # 영화 제목
                elif index ==6:
                    print(f'runtime :  {td.get_text(strip=True)}') # 상영시간 

            print('@@@@@@@@@@@@@@@@@@@@@@@@@')
            
        print(f'{page}@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
            

        

#     print(f'{page}@{page}@{page}@{page}@{page}@{page}@{page}@{page}@{page}@{page}@{page}@{page}@{page}@{page}@{page}@{page}@{page}@')


def main_movie_crawler():
    
    # 오늘 날짜 먼저 가져옴 
    now = datetime.datetime.now()
    year = now.year
    day = now.day
    month = now.month

    
#     ##### 동대문구 크롤러##### 

#     dongdaemungu_area_code_list = ['MA','MF','MB','MC','MJ'] 
#     # 정보화,답십리, 장안, 용두, 휘셩 (이문 제외)
# #     dongdaemungu_area_code_list = ['MA']
    
#     for libGroup in dongdaemungu_area_code_list:
#         dongdaemungu_movie_crawler(libGroup)
#         print(f'{libGroup} #####################################################################################################################################')


#     ##### 성동구 크롤러 #####
    
#     seongdonggu_area_code_list = ['SD','YD','SS','KH','CG'] 
#     # 성동구립,용답, 장안, 성수, 금호,청계
# #     seongdonggu_area_code_list = ['SD']
    
#     for area_code in seongdonggu_area_code_list:
#         seongdonggu_movie_crawler(area_code,year)
#         print(f'{area_code}#############################################################################################################################')


    ##### 광진구 크롤러 #####
    



    

In [71]:
main_movie_crawler()

time_date : 2018-11-03
time_hour : 10:00
title : 코코
runtime :  127분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-10-31
time_hour : 15:00
title : 히 러브스 미
runtime :  91분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-10-28
time_hour : 11:00
title : 리오 2
runtime :  101분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-10-28
time_hour : 15:00
title : 레고무비- 플래시
runtime :  78분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-10-27
time_hour : 10:00
title : 스파키
runtime :  91분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-10-26
time_hour : 10:00
title : 땐뽀걸즈
runtime :  85분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-10-25
time_hour : 15:00
title : 굿바이 크리스토퍼 로빈
runtime :  107분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-10-24
time_hour : 15:00
title : 채비
runtime :  114분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-10-21
time_hour : 11:00
title : 비포와 친구들
runtime :  82분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-10-21
time_hour : 15:00
title : 무민가족의 한여름 대소동
runtime :  73분
@@@@@@@@@@@@@@@@@@@@@@@@@
1@@@@@@@@@@@@@@@@@@@@@@

time_date : 2017-12-17
time_hour : 11:00
title : 샤크
runtime :  90분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2017-12-10
time_hour : 11:00
title : 마이펫 오지
runtime :  89분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2017-12-03
time_hour : 15:00
title : 빅 히어로
runtime :  112분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2017-11-26
time_hour : 15:00
title : 아이스에이지 5
runtime :  94분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2017-11-19
time_hour : 15:00
title : 고양이의 보은
runtime :  75분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2017-11-12
time_hour : 15:00
title : 마다가스카 3
runtime :  93분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2017-11-05
time_hour : 15:00
title : 아기배달부 스토크
runtime :  87분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2017-10-29
time_hour : 11:00
title : 리틀 프린세스 소피아-비밀의 도서관
runtime :  99분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2017-10-22
time_hour : 15:00
title : 호비와 쿠우의 어드벤처
runtime :  60분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2017-10-15
time_hour : 11:00
title : 우주로봇 씨어
runtime :  89분
@@@@@@@@@@@@@@@@@@@@@@@@@
5@@@@@@

time_date : 2018-07-14
time_hour : 15:00
title : 토르: 마법망치의 전설
runtime :  83분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-07-08
time_hour : 15:00
title : 더 매직: 리틀톰과 도둑공주
runtime :  78분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-07-07
time_hour : 15:00
title : 제이크 선장과 네버랜드 해적들
runtime :  117분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-07-01
time_hour : 15:00
title : 일곱 난쟁이
runtime :  87분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-06-30
time_hour : 15:00
title : 레고 배트맨 무비
runtime :  104분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-06-24
time_hour : 15:00
title : 슈퍼배드 2
runtime :  98분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-06-23
time_hour : 15:00
title : 슈퍼배드 1
runtime :  95분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-06-17
time_hour : 15:00
title : 삼총사: 용감한 친구들
runtime :  77분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-06-16
time_hour : 15:00
title : 하늘에서 음식이 내린다면 2
runtime :  95분
@@@@@@@@@@@@@@@@@@@@@@@@@
time_date : 2018-06-10
time_hour : 15:00
title : 볼트
runtime :  96분
@@@@@@@@@@@@@@@